# Week 4
# day3: 31 Aug 2022

4. Learn about Gradient Descent and its below variants:

    Momentum

    Nesterov

    Adagrad

    RMSProp

    Adam
5. Implement all the above in Numpy
6. How does the "Exponential weighted average" lecture given in the readings   relate to some of the variants of Gradient Descent?

In [5]:
import numpy as np

In [6]:
x = np.array([1.0, 3.0, 7.0, 2.0, 5.0, 4.0])
y = np.array([4.0, 9.0, 20.0, 7.0, 15.0, 11.0])


def linear_regression(x, m, b):
    yhat = x * m + b
    return yhat

## RMSprop

In [38]:
x = np.array(
    [
        1.0,
        0.0,
        0.0,
        3.0,
        0.0,
        0.0,
        0.0,
        7.0,
        0.0,
        2.0,
        5.0,
        0.0,
        0.0,
        0.0,
        0.0,
        4.0,
        0.0,
        0.0,
        0.0,
    ]
)
y = np.array(
    [
        4.0,
        0.0,
        -1.0,
        9.0,
        0.0,
        0.0,
        0.0,
        20.0,
        0.0,
        7.0,
        15.0,
        0.0,
        0.1,
        0.0,
        0.0,
        11.0,
        0.0,
        1.0,
        0.0,
    ]
)

In [60]:
x = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        0.0,
        0.0,
        3.0,
        0.0,
        0.0,
        0.0,
        7.0,
        0.0,
        2.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        5.0,
        0.0,
        0.0,
        0.0,
        0.0,
        4.0,
        0.0,
        0.0,
        0.0,
    ]
)
y = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        4.0,
        0.0,
        -1.0,
        9.0,
        0.0,
        0.0,
        0.0,
        20.0,
        0.0,
        7.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        15.0,
        0.0,
        0.1,
        0.0,
        0.0,
        11.0,
        0.0,
        1.0,
        0.0,
    ]
)
len(y)

35

In [61]:
# RMSProp
class RMSProp:
    def __init__(self):
        self.learning_rate = 0.1
        self.w = 9
        self.b = 0
        self.update_w = 0
        self.update_b = 0
        self.epsilon = 1e-6
        self.beta = 0.9

    def update_weight(self, x, y):
        yhat = self.w * x + self.b
        self.update_w = self.beta * self.update_w + (1 - self.beta) * (
            (-2 * sum(x * (y - yhat)).mean()) ** 2
        )
        self.update_b = self.beta * self.update_b + (1 - self.beta) * (
            (-2 * sum(y - yhat).mean()) ** 2
        )
        self.w -= (self.learning_rate / np.sqrt(self.update_w + self.epsilon)) * (
            -2 * sum(x * (y - yhat)).mean()
        )
        self.b -= (self.learning_rate / np.sqrt(self.update_b + self.epsilon)) * (
            -2 * sum(y - yhat).mean()
        )

    def MSE(self, y, yhat):
        return np.square(np.subtract(y, yhat)).mean()

    def fit(self, x, y, epochs=2000):
        history = []
        for e in range(epochs):
            self.update_weight(x, y)
            loss = self.MSE(y, (self.w * x + self.b))
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss}")
                print(f"weight:{self.w},bias:{self.b}")
            history.append(loss)
            if loss <= 1:
                print(f"Epoch: {e}, Loss: {loss}")
                print(f"weight:{self.w},bias:{self.b}")
                return history
        return history

In [62]:
rms = RMSProp()
history = rms.fit(x, y)
print("Prediction:")
print(rms.w, rms.b)
print("yhat:", linear_regression(x, rms.w, rms.b))

Epoch: 0, Loss: 96.6050634024521
weight:8.683772233984152,bias:-0.31622776599411734
Epoch: 63, Loss: 0.8798530692190021
weight:3.4413234141001836,bias:-0.32961164219216027
Prediction:
3.4413234141001836 -0.32961164219216027
yhat: [-0.32961164 -0.32961164 -0.32961164 -0.32961164 -0.32961164 -0.32961164
 -0.32961164 -0.32961164  3.11171177 -0.32961164 -0.32961164  9.9943586
 -0.32961164 -0.32961164 -0.32961164 23.75965226 -0.32961164  6.55303519
 -0.32961164 -0.32961164 -0.32961164 -0.32961164 -0.32961164 -0.32961164
 -0.32961164 -0.32961164 16.87700543 -0.32961164 -0.32961164 -0.32961164
 -0.32961164 13.43568201 -0.32961164 -0.32961164 -0.32961164]


In [73]:
# Adam
class Adam:
    def __init__(self):
        self.learning_rate = 0.1
        self.w = 9
        self.b = 0
        self.update_m_w = 0
        self.update_m_b = 0
        self.update_v_w = 0
        self.update_v_b = 0
        self.epsilon = 1e-8
        self.beta1 = 0.9
        self.beta2 = 0.99

    def update_weight(self, x, y, e):
        yhat = self.w * x + self.b
        self.update_m_w = self.beta1 * self.update_m_w + (1 - self.beta1) * (
            -2 * sum(x * (y - yhat)).mean()
        )
        self.update_m_b = self.beta1 * self.update_m_b + (1 - self.beta1) * (
            -2 * sum(y - yhat).mean()
        )
        self.update_v_w = self.beta2 * self.update_v_w + (1 - self.beta2) * (
            (-2 * sum(x * (y - yhat)).mean()) ** 2
        )
        self.update_v_b = self.beta2 * self.update_v_b + (1 - self.beta2) * (
            (-2 * sum(y - yhat).mean()) ** 2
        )
        self.update_m_w = self.update_m_w / (1 - self.beta1**e + 1)
        self.update_m_b = self.update_m_b / (1 - self.beta1**e + 1)
        self.update_v_w = self.update_v_w / (1 - self.beta2**e + 1)
        self.update_v_b = self.update_v_b / (1 - self.beta2**e + 1)
        self.w -= (self.learning_rate / np.sqrt(self.update_v_w + self.epsilon)) * (
            self.update_m_w
        )
        self.b -= (self.learning_rate / np.sqrt(self.update_v_b + self.epsilon)) * (
            self.update_m_b
        )

    def MSE(self, y, yhat):
        return np.square(np.subtract(y, yhat)).mean()

    def fit(self, x, y, epochs=2000):
        history = []
        for e in range(epochs):
            self.update_weight(x, y, e)
            loss = self.MSE(y, (self.w * x + self.b))
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss}")
                print(f"weight:{self.w},bias:{self.b}")
            history.append(loss)
            if loss <= 1:
                print(f"Epoch: {e}, Loss: {loss}")
                print(f"weight:{self.w},bias:{self.b}")
                return history
        return history

In [74]:
adam = Adam()
history = adam.fit(x, y)
print("Prediction:")
print(adam.w, adam.b)
print("yhat:", linear_regression(x, adam.w, adam.b))

Epoch: 0, Loss: 105.57314285714911
weight:8.90000000000003,bias:-0.09999999999928146
Epoch: 94, Loss: 0.9592518594789811
weight:3.4706544458413977,bias:-0.29144372373459765
Prediction:
3.4706544458413977 -0.29144372373459765
yhat: [-0.29144372 -0.29144372 -0.29144372 -0.29144372 -0.29144372 -0.29144372
 -0.29144372 -0.29144372  3.17921072 -0.29144372 -0.29144372 10.12051961
 -0.29144372 -0.29144372 -0.29144372 24.0031374  -0.29144372  6.64986517
 -0.29144372 -0.29144372 -0.29144372 -0.29144372 -0.29144372 -0.29144372
 -0.29144372 -0.29144372 17.06182851 -0.29144372 -0.29144372 -0.29144372
 -0.29144372 13.59117406 -0.29144372 -0.29144372 -0.29144372]


6. How does the "Exponential weighted average" lecture given in the readings   relate to some of the variants of Gradient Descent?

Weight updates are guided by the previous gradients.The most recent gradient provide more information so the older the gradient the lesser the weight should be given thus exponential weightd average is used 